In [ ]:
$ ansible-config dump|grep roles
DEFAULT_ROLES_PATH(default) = ['/home/ubuntu/.ansible/roles', '/usr/share/ansible/roles', '/etc/ansible/roles']


In [ ]:
$ mkdir -p /home/ubuntu/.ansible/roles
$ cd /home/ubuntu/.ansible/roles

In [ ]:
$ ansible-galaxy init lemp

- Role lemp was created successfully
ubuntu@ip-172-31-37-145:~/.ansible/roles$ ll
total 12
drwxrwxr-x 3 ubuntu ubuntu 4096 Jul 10 17:46 ./
drwx------ 4 ubuntu ubuntu 4096 Jul 10 17:43 ../
drwxrwxr-x 8 ubuntu ubuntu 4096 Jul 10 17:46 lemp/
ubuntu@ip-172-31-37-145:~/.ansible/roles$ tree lemp/
lemp/
├── README.md
├── defaults
│   └── main.yml
├── handlers
│   └── main.yml
├── meta
│   └── main.yml
├── tasks
│   └── main.yml
├── tests
│   ├── inventory
│   └── test.yml
└── vars
    └── main.yml

6 directories, 8 files


In [ ]:
$ cd lemp/



## Variables

In [ ]:
# vars/main.yml and defaults/main.yml are used for declaring variables and /vars/main.yml have more priority.

In [ ]:
$ vim vars/main.yml 

---
packages:
  - nginx
  - php8.1-fpm 
  - php8.1 
  - php8.1-common 
  - php8.1-mysql 
  - php8.1-xml 
  - php8.1-xmlrpc 
  - php8.1-curl 
  - php8.1-gd 
  - php8.1-imagick 
  - php8.1-cli 
  - php8.1-imap 
  - php8.1-mbstring 
  - php8.1-opcache 
  - php8.1-soap 
  - php8.1-zip 
  - php8.1-intl 
  - php8.1-bcmath 
  - unzip

nginx_port: "8080"
nginx_user: "www-data"
nginx_group: "www-data"
domain_name: "nginx.inenso.in"

maria_root_pwd: "mysqlroot@123*"
maria_user: "wordpressuser"
maria_db: "wordpresssdb"
maria_pwd: "wordpresspwd"


## template

In [ ]:
$ vim templates/vhost.tmpl

server {
        listen {{ nginx_port }};
        listen [::]:{{ nginx_port }};
        root /var/www/{{ domain_name }};
        index index.php index.html index.htm;
        server_name {{ domain_name }};
 
        location / {
            try_files $uri $uri/ =404;
        }
 
        location ~ \.php$ {
            include snippets/fastcgi-php.conf;
            fastcgi_pass unix:/var/run/php/php8.1-fpm.sock;
        }
}


## handlers

In [ ]:
$ vim handlers/main.yml


---
- name: "restart/enable-nginx-php8.1-fpm"
  service:
    name: "{{ item }}"
    enabled: yes
    state: restarted
  with_items:
    - nginx
    - php8.1-fpm


## files

In [ ]:
ubuntu@ip-172-31-37-145:~/.ansible/roles/lemp$ mkdir files
$ echo '<h1><center> This is Ansible  Nginx-Wordpress </h1></center> ' 
<h1><center> This is Ansible  Nginx-Wordpress </h1></center> 

$ echo '<h1><center> This is Ansible  Nginx-Wordpress </h1></center> ' > /home/ubuntu/.ansible/roles/lemp/files/info.html


$ tree /home/ubuntu/.ansible/roles/lemp/
/home/ubuntu/.ansible/roles/lemp/
├── README.md
├── defaults
│   └── main.yml
├── files
│   └── info.php
├── handlers
│   └── main.yml
├── meta
│   └── main.yml
├── tasks
│   └── main.yml
├── templates
│   └── vhost.tmpl
├── tests
│   ├── inventory
│   └── test.yml
└── vars
    └── main.yml

8 directories, 10 files


## tasks

In [ ]:
$ cat tasks/main.yml 
---
- name: " Installing mariadb-server and python3-pip"
  apt:
    name:
      - mariadb-server
      - python3-pip
    state: present
    update_cache: yes

- name: "restart/enable Mariadb-server"
  service:
    name: mariadb
    state: restarted
    enabled: yes

- name: "Python connector/module Installation"
  pip:
    name: PyMySQL
    state: present

- name: "Resetting Mariadb root Password"
  ignore_errors: yes
  no_log: true
  mysql_user:
    host_all: yes
    login_unix_socket: /run/mysqld/mysqld.sock
    login_password: ""
    login_user: root
    name: root
    password: "{{ maria_root_pwd }}"

- name: "Creating Mariadb database for wordpress"
  mysql_db:
    login_password: "{{ maria_root_pwd }}"
    login_unix_socket: /run/mysqld/mysqld.sock
    login_user: root
    name: "{{ maria_db }}"
    state: present

- name: "Creating mariadb user for wp with privilege"
  no_log: true
  mysql_user:
    host: "%"
    login_password: "{{ maria_root_pwd }}"
    login_unix_socket: /run/mysqld/mysqld.sock
    login_user: root
    name: "{{ maria_user }}"
    password: "{{ maria_pwd }}"
    priv: "{{ maria_db }}.*:ALL"
    state: present

- name: "Nginx and php8.1-fpm and extensions Installation"
  apt:
    name: "{{ packages }}"
    state: present
    update_cache: yes
  register: pkg_status

- name: "restart/enable-nginx-php8.1-fpm"
  when: pkg_status.changed == true
  service:
    enabled: yes
    name: "{{ item }}"
    state: restarted
  with_items:
    - nginx
    - php8.1-fpm

- name: "Creating Document root /var/www/{{ domain_name}}"
  file:
    group: "{{ nginx_group }}"
    owner: "{{ nginx_user }}"
    path: "/var/www/{{ domain_name }}"
    state: directory

- name: "Creating nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf"
  template:
    dest: "/etc/nginx/sites-available/{{ domain_name }}.conf"
    src: vhost.tmpl

- name: "Enabling nginx conf /etc/nginx/sites-available/{{ domain_name }}.conf"
  file:
    src: "/etc/nginx/sites-available/{{ domain_name }}.conf"
    state: link
    path: "/etc/nginx/sites-enabled/{{ domain_name }}.conf"
  notify:
    - restart/enable-nginx-php8.1-fpm


- name: "Copying info.php file"
  copy:
    dest: "/var/www/{{ domain_name }}"
    src: "{{ item }}"
    group: "{{ nginx_group }}"
    owner: "{{ nginx_user }}"
  with_items:
    - info.php
    - info.html

## wordpress Installation using role

In [ ]:
$ cat hosts 
[ubuntu]

52.66.213.107  ansible_user="ubuntu" ansible_port=22 ansible_ssh_private_key_file="ansible.pem"

In [ ]:
$ cat wp-config-sample.php 


<?php
/**
 * The base configuration for WordPress
 *
 * The wp-config.php creation script uses this file during the installation.
 * You don't have to use the web site, you can copy this file to "wp-config.php"
 * and fill in the values.
 *
 * This file contains the following configurations:
 *
 * * Database settings
 * * Secret keys
 * * Database table prefix
 * * ABSPATH
 *
 * @link https://wordpress.org/support/article/editing-wp-config-php/
 *
 * @package WordPress
 */

// ** Database settings - You can get this info from your web host ** //
/** The name of the database for WordPress */

define( 'WP_HOME', 'http://{{ domain_name }}' );
define( 'WP_SITEURL', 'http://{{ domain_name }}' );

define( 'DB_NAME', '{{ maria_db }}' );

/** Database username */
define( 'DB_USER', '{{ maria_user }}' );

/** Database password */
define( 'DB_PASSWORD', '{{ maria_pwd }}' );

/** Database hostname */
define( 'DB_HOST', 'localhost' );


/** Database charset to use in creating database tables. */
define( 'DB_CHARSET', 'utf8' );

/** The database collate type. Don't change this if in doubt. */
define( 'DB_COLLATE', '' );

/**#@+
 * Authentication unique keys and salts.
 *
 * Change these to different unique phrases! You can generate these using
 * the {@link https://api.wordpress.org/secret-key/1.1/salt/ WordPress.org secret-key service}.
 *
 * You can change these at any point in time to invalidate all existing cookies.
 * This will force all users to have to log in again.
 *
 * @since 2.6.0
 */
define( 'AUTH_KEY',         'put your unique phrase here' );
define( 'SECURE_AUTH_KEY',  'put your unique phrase here' );
define( 'LOGGED_IN_KEY',    'put your unique phrase here' );
define( 'NONCE_KEY',        'put your unique phrase here' );
define( 'AUTH_SALT',        'put your unique phrase here' );
define( 'SECURE_AUTH_SALT', 'put your unique phrase here' );
define( 'LOGGED_IN_SALT',   'put your unique phrase here' );
define( 'NONCE_SALT',       'put your unique phrase here' );

/**#@-*/

/**
 * WordPress database table prefix.
 *
 * You can have multiple installations in one database if you give each
 * a unique prefix. Only numbers, letters, and underscores please!
 */
$table_prefix = 'wp_';

/**
 * For developers: WordPress debugging mode.
 *
 * Change this to true to enable the display of notices during development.
 * It is strongly recommended that plugin and theme developers use WP_DEBUG
 * in their development environments.
 *
 * For information on other constants that can be used for debugging,
 * visit the documentation.
 *
 * @link https://wordpress.org/support/article/debugging-in-wordpress/
 */
define( 'WP_DEBUG', false );

/* Add any custom values between this line and the "stop editing" line. */



/* That's all, stop editing! Happy publishing. */

/** Absolute path to the WordPress directory. */
if ( ! defined( 'ABSPATH' ) ) {
	define( 'ABSPATH', __DIR__ . '/' );
}

/** Sets up WordPress vars and included files. */
require_once ABSPATH . 'wp-settings.php';

In [ ]:
# vim wordpress-nginx-ubuntu22.04

---
- name: "wordpress-nginx-ubuntu22.04 using role"
  hosts: ubuntu 
  become: yes 
  vars:
    wp_url: "https://wordpress.org/latest.tar.gz"
  
  roles:
    - lemp
  tasks:
    - name: "Downloading wordpress"
      get_url:
        dest: /tmp/wordpress.tar.gz
        url: "{{ wp_url }}"

    - name: "Extracting wordpress files"
      unarchive:
        dest: /tmp/
        remote_src: yes
        src: /tmp/wordpress.tar.gz

    - name: "Copying wordpress files to document root /var/www/{{ domain_name }}"
      copy:
        remote_src: yes
        src: "/tmp/wordpress/"
        dest: "/var/www/{{ domain_name }}/"
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"
      notify:
        - restart/enable-nginx-php8.1-fpm

    - name: "copying/updating wp-config.php"
      template:
        dest: "/var/www/{{ domain_name }}/wp-config.php"
        src: wp-config-sample.php
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"
      notify:
        - restart/enable-nginx-php8.1-fpm
  
    - name: "cleanup"
      file:
        state: absent
        path: "{{ item }}"
      with_items:
        - /tmp/wordpress
        - /tmp/wordpress.tar.gz
    
    

In [ ]:
$ ansible-playbook -i hosts wordpress-nginx-ubuntu22.04.yml 

PLAY [wordpress-nginx-ubuntu22.04 using role] *************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [52.66.213.107]

TASK [lemp :  Installing mariadb-server and python3-pip] **************************************************************************************************
ok: [52.66.213.107]

TASK [lemp : restart/enable Mariadb-server] ***************************************************************************************************************
changed: [52.66.213.107]

TASK [lemp : Python connector/module Installation] ********************************************************************************************************
ok: [52.66.213.107]

TASK [lemp : Resetting Mariadb root Password] *************************************************************************************************************
fatal: [52.66.213.107]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [lemp : Creating Mariadb database for wordpress] *****************************************************************************************************
ok: [52.66.213.107]

TASK [lemp : Creating mariadb user for wp with privilege] *************************************************************************************************
ok: [52.66.213.107]

TASK [lemp : Nginx and php8.1-fpm and extensions Installation] ********************************************************************************************
ok: [52.66.213.107]

TASK [lemp : restart/enable-nginx-php8.1-fpm] *************************************************************************************************************
skipping: [52.66.213.107] => (item=nginx) 
skipping: [52.66.213.107] => (item=php8.1-fpm) 

TASK [lemp : Creating Document root /var/www/nginx.inenso.in] *********************************************************************************************
ok: [52.66.213.107]

TASK [lemp : Creating nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] *************************************************************************
ok: [52.66.213.107]

TASK [lemp : Enabling nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] *************************************************************************
ok: [52.66.213.107]

TASK [lemp : Copying info.php file] ***********************************************************************************************************************
ok: [52.66.213.107] => (item=info.php)
ok: [52.66.213.107] => (item=info.html)

TASK [Downloading wordpress] ******************************************************************************************************************************
ok: [52.66.213.107]

TASK [Extracting wordpress files] *************************************************************************************************************************
ok: [52.66.213.107]

TASK [Copying wordpress files to document root /var/www/nginx.inenso.in] **********************************************************************************
ok: [52.66.213.107]

TASK [copying/updating wp-config.php] *********************************************************************************************************************
changed: [52.66.213.107]

TASK [cleanup] ********************************************************************************************************************************************
changed: [52.66.213.107] => (item=/tmp/wordpress)
changed: [52.66.213.107] => (item=/tmp/wordpress.tar.gz)

RUNNING HANDLER [lemp : restart/enable-nginx-php8.1-fpm] **************************************************************************************************
changed: [52.66.213.107] => (item=nginx)
changed: [52.66.213.107] => (item=php8.1-fpm)

PLAY RECAP ************************************************************************************************************************************************
52.66.213.107              : ok=18   changed=4    unreachable=0    failed=0    skipped=1    rescued=0    ignored=1 

In [ ]:
$ ansible-playbook -i hosts wordpress-nginx-ubuntu22.04.yml 

PLAY [wordpress-nginx-ubuntu22.04 using role] *************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [52.66.213.107]

TASK [lemp :  Installing mariadb-server and python3-pip] **************************************************************************************************
ok: [52.66.213.107]

TASK [lemp : restart/enable Mariadb-server] ***************************************************************************************************************
changed: [52.66.213.107]

TASK [lemp : Python connector/module Installation] ********************************************************************************************************
ok: [52.66.213.107]

TASK [lemp : Resetting Mariadb root Password] *************************************************************************************************************
fatal: [52.66.213.107]: FAILED! => {"censored": "the output has been hidden due to the fact that 'no_log: true' was specified for this result", "changed": false}
...ignoring

TASK [lemp : Creating Mariadb database for wordpress] *****************************************************************************************************
ok: [52.66.213.107]

TASK [lemp : Creating mariadb user for wp with privilege] *************************************************************************************************
ok: [52.66.213.107]

TASK [lemp : Nginx and php8.1-fpm and extensions Installation] ********************************************************************************************
ok: [52.66.213.107]

TASK [lemp : restart/enable-nginx-php8.1-fpm] *************************************************************************************************************
skipping: [52.66.213.107] => (item=nginx) 
skipping: [52.66.213.107] => (item=php8.1-fpm) 

TASK [lemp : Creating Document root /var/www/nginx.inenso.in] *********************************************************************************************
ok: [52.66.213.107]

TASK [lemp : Creating nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] *************************************************************************
ok: [52.66.213.107]

TASK [lemp : Enabling nginx conf /etc/nginx/sites-available/nginx.inenso.in.conf] *************************************************************************
ok: [52.66.213.107]

TASK [lemp : Copying info.php file] ***********************************************************************************************************************
ok: [52.66.213.107] => (item=info.php)
ok: [52.66.213.107] => (item=info.html)

TASK [Downloading wordpress] ******************************************************************************************************************************
changed: [52.66.213.107]

TASK [Extracting wordpress files] *************************************************************************************************************************
changed: [52.66.213.107]

TASK [Copying wordpress files to document root /var/www/nginx.inenso.in] **********************************************************************************
ok: [52.66.213.107]

TASK [copying/updating wp-config.php] *********************************************************************************************************************
ok: [52.66.213.107]

TASK [cleanup] ********************************************************************************************************************************************
changed: [52.66.213.107] => (item=/tmp/wordpress)
changed: [52.66.213.107] => (item=/tmp/wordpress.tar.gz)

PLAY RECAP ************************************************************************************************************************************************
52.66.213.107              : ok=17   changed=4    unreachable=0    failed=0    skipped=1    rescued=0    ignored=1 